In [12]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging

In [34]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
from datasets import load_dataset

dataset = load_dataset("dutch_social")

Reusing dataset dutch_social (C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset['test']


Dataset({
    features: ['full_text', 'text_translation', 'screen_name', 'description', 'desc_translation', 'location', 'weekofyear', 'weekday', 'month', 'year', 'day', 'point_info', 'point', 'latitude', 'longitude', 'altitude', 'province', 'hisco_standard', 'hisco_code', 'industry', 'sentiment_pattern', 'subjective_pattern', 'label'],
    num_rows: 54268
})

In [5]:
test_data = pd.DataFrame(dataset['test'])
train_data = pd.DataFrame(dataset['train'])
validation_data = pd.DataFrame(dataset['validation'])

In [6]:
train_data.head()

,full_text,text_translation,screen_name,description,desc_translation,location,weekofyear,weekday,month,year,...,latitude,longitude,altitude,province,hisco_standard,hisco_code,industry,sentiment_pattern,subjective_pattern,label
0,"Maar , er iets nuttigs mee doen ? Zie jij 'm v...","However, there is something useful to do with ...",RonaldMeeuwis,None,None,None,21,3,5,2020,...,0.000000,0.000000,0.0,False,None,None,False,0.0,0.0,1
1,RT @daphneskopelos: Door de coronacrisis zijn ...,RT @daphneskopelos: The corona crisis are abou...,IBeugel,"Journalist, programmamaker, schrijver, oud Bal...","Journalist, filmmaker, writer, former Balkans ...",Griekenland,23,2,6,2020,...,38.995368,21.987713,0.0,False,None,None,False,0.0,0.0,1
2,RT @Geovation: What role do ethics and locatio...,RT @Geovation: What role do ethics play data a...,hanscees,"Systeemdenker, eigenaar https://t.co/5Cgd9GwmW...","Systems Thinker, owner https://t.co/5Cgd9GwmWt...",Netherlands,18,4,5,2020,...,52.500170,5.748082,0.0,Flevoland,None,None,False,0.0,0.0,1
3,RT @MollyJongFast: Laura Ingraham is going to ...,RT @MollyJongFast: Laura Ingraham is going to ...,LDUniGr,CSO CarbExplore BV https://t.co/8IoRT28pWm\nem...,CSO CarbExplore BV https://t.co/8IoRT28pWm\nem...,"Groningen, The Netherlands",21,0,5,2020,...,53.219065,6.568008,0.0,Groningen,None,None,False,0.0,0.0,1
4,RT @kneeyockartee: Corona has robbed me of man...,RT @kneeyockartee: Corona has robbed me of man...,transxlucence,welkom in de chaos van mijn hoofd. geniet van ...,welcome to the chaos of my head. enjoy the con...,amsterdam/utrecht,37,6,9,2020,...,52.094975,5.109708,0.0,Utrecht,None,None,False,0.0,0.0,1


In [9]:
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features


featuresets = [(dialogue_act_features(post), target) for post, target in list(zip(train_data['full_text'],train_data['label']))]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.6014127764127765


In [48]:
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base", padding = True)
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels = 3)

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.dense.weight', 'class